In [0]:
%run "../Configuration"

In [0]:
# Define the metadata widget:
dbutils.widgets.text("entityName", "")
dbutils.widgets.text("entityIngestionColumns", "")
dbutils.widgets.text("sourceSystem", "")

# Save parameters to variables:
entityName =       dbutils.widgets.get('entityName')
sourceSystemName = dbutils.widgets.get('sourceSystem')
columns =          json.loads(dbutils.widgets.get('entityIngestionColumns'))
raw_path =         entity_bronze(entityName)

In [0]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(BRONZE + raw_path)

In [0]:
df.createOrReplaceTempView(f'tv_{entityName}')

In [0]:
# Cast data types:
df = cast_data_types_transformation(columns, df)

# Drop unwanted columns:
df = drop_unwanted_columns_transformation(columns, df)

# Drop duplicate values:
df = drop_duplicates_transformation(df)

In [0]:
# Check data exists:
if df.count() > 0:
    # Set location for delta table:
    location = f"/{sourceSystemName}/{entityName}/{date_path}/"
    
    # Write to delta table:
    df.write.mode('overwrite').format('delta').option('header', 'true').save(SILVER + location)

    # Output location:
    dbutils.notebook.exit(location)
else:
    dbutils.notebook.exit(-1)